In [ ]:
%cd /content/drive/MyDrive/Makaki_RecSys

/content/drive/.shortcut-targets-by-id/1YEXu_mHNPxVkZx_LwyiucQWcTYXGhyLN/Makaki_RecSys


In [ ]:
import pandas as pd
from pandas import read_parquet, DataFrame, to_datetime, concat
qp = lambda p: {'name': f'q{p}'.replace('0.',''), 'func': lambda x: x.quantile(q=p)}

In [ ]:
#df_smm = pd.read_parquet('train_smm.parquet')
df_zvuk = read_parquet('train_zvuk.parquet')
#df_smm_test = pd.read_parquet('test_smm.parquet')
#df_zvuk_test = pd.read_parquet('test_zvuk.parquet')

In [ ]:
def feature_groupby(
    data: DataFrame,
    group_column: str,
    column: list,
    agg_func: str
    ) -> DataFrame:
  all_column = [group_column, *column]
  df = data[all_column].copy()
  if type(agg_func) == dict:
    new_name = [f'{group_column}_{col}_{agg_func["name"]}' for col in column]
    df[new_name] = df.groupby([group_column])[column].transform(agg_func['func'])
  elif type(agg_func) == str:
    new_name = [f'{group_column}_{col}_{agg_func}' for col in column]
    df[new_name] = df.groupby([group_column])[column].transform(agg_func)
  return df.drop(all_column, axis=1)

In [ ]:
def timestamp_feature(df: DataFrame) -> DataFrame:
  df.timestamp = to_datetime(df.timestamp)
  df['week'] = df.timestamp.dt.isocalendar().week
  df['day_of_week'] = df.timestamp.dt.weekday
  df['hour'] = df.timestamp.dt.hour

  mode_user = feature_groupby(
      df, 'user_id', ['hour', 'day_of_week', 'week'], {'name': 'mode', 'func':lambda x: x.mode()[0]}
      )
  mode_item = feature_groupby(
      df, 'item_id', ['hour', 'day_of_week', 'week'], {'name': 'mode', 'func':lambda x: x.mode()[0]}
      )
  df = concat([df, mode_user, mode_item], axis=1)

  agg_arr = ['max', 'mean']
  for agg_func in agg_arr:
    agg_user = feature_groupby(df, 'user_id', ['hour', 'day_of_week'], agg_func)
    agg_item = feature_groupby(df, 'item_id', ['hour', 'day_of_week'], agg_func)
    df = concat([df, agg_user, agg_item], axis=1)
  return df.drop(['hour', 'day_of_week', 'week'], axis=1)

def timestamp_feature(df: DataFrame, df_pred: DataFrame) -> DataFrame:
  df.timestamp = to_datetime(df.timestamp)
  df['week'] = df.timestamp.dt.isocalendar().week
  df['day_of_week'] = df.timestamp.dt.weekday
  df['hour'] = df.timestamp.dt.hour

  mode_user = feature_groupby(
      df, 'user_id', ['hour', 'day_of_week', 'week'], {'name': 'mode', 'func':lambda x: x.mode()[0]}
      )
  mode_item = feature_groupby(
      df, 'item_id', ['hour', 'day_of_week', 'week'], {'name': 'mode', 'func':lambda x: x.mode()[0]}
      )
  df = concat([df, mode_user, mode_item], axis=1)

  agg_arr = ['max', 'mean']
  for agg_func in agg_arr:
    agg_user = feature_groupby(df, 'user_id', ['hour', 'day_of_week'], agg_func)
    agg_item = feature_groupby(df, 'item_id', ['hour', 'day_of_week'], agg_func)
    df = concat([df, agg_user, agg_item], axis=1)
  return df.drop(['hour', 'day_of_week', 'week'], axis=1)

In [ ]:
def feature_groupby(
    data: DataFrame,
    group_column: str,
    column: list,
    agg_func: str
    ) -> DataFrame:
  all_column = [group_column, *column]
  df = data[all_column].copy()
  if type(agg_func) == dict:
    new_name = [f'{group_column}_{col}_{agg_func["name"]}' for col in column]
    df[new_name] = df.groupby([group_column])[column].transform(agg_func['func'])
  elif type(agg_func) == str:
    new_name = [f'{group_column}_{col}_{agg_func}' for col in column]
    df[new_name] = df.groupby([group_column])[column].transform(agg_func)
  return df.drop(all_column, axis=1)

def timestamp_feature(df: DataFrame) -> DataFrame:
  df.timestamp = to_datetime(df.timestamp)
  df['week'] = df.timestamp.dt.isocalendar().week
  df['day_of_week'] = df.timestamp.dt.weekday
  df['hour'] = df.timestamp.dt.hour

  mode_user = feature_groupby(
      df, 'user_id', ['hour', 'day_of_week', 'week'], {'name': 'mode', 'func':lambda x: x.mode()[0]}
      )
  mode_item = feature_groupby(
      df, 'item_id', ['hour', 'day_of_week', 'week'], {'name': 'mode', 'func':lambda x: x.mode()[0]}
      )
  df = concat([df, mode_user, mode_item], axis=1)

  agg_arr = ['max', 'mean']
  for agg_func in agg_arr:
    agg_user = feature_groupby(df, 'user_id', ['hour', 'day_of_week'], agg_func)
    agg_item = feature_groupby(df, 'item_id', ['hour', 'day_of_week'], agg_func)
    df = concat([df, agg_user, agg_item], axis=1)
  return df.drop(['hour', 'day_of_week', 'week'], axis=1)

def rating_feature(df: DataFrame) -> DataFrame:
  group_columns = ['user_id', 'item_id']
  agg_arr = ['max', 'mean', qp(0.90), qp(0.95), qp(0.99)]
  for agg_func in agg_arr:
    for group in group_columns:
      agg_ = feature_groupby(df, group, ['rating'], agg_func)
      df = concat([df, agg_], axis=1)
  return df

def freq_item_feature(df: DataFrame) -> DataFrame:
  freq_item = df['item_id'].value_counts(normalize=True).rename('freq_item')
  df = df.join(freq_item, on = 'item_id', how = 'inner')

  agg_arr = ['max', 'mean', 'median', qp(0.25), qp(0.75)]
  for agg_func in agg_arr:
    agg_ = feature_groupby(df, 'user_id', ['freq_item'], agg_func)
    df = concat([df, agg_], axis=1)
  return df

In [ ]:
def freq_item_feature(df: DataFrame) -> DataFrame:
  freq_item = df['item_id'].value_counts(normalize=True).rename('freq_item')
  df = df.join(freq_item, on = 'item_id', how = 'inner')

  agg_arr = ['max', 'mean', 'median', qp(0.25), qp(0.75)]
  for agg_func in agg_arr:
    agg_ = feature_groupby(df, 'user_id', ['freq_item'], agg_func)
    df = concat([df, agg_], axis=1)
  return df

In [ ]:
df_zvuk = freq_item_feature(df_zvuk)

In [ ]:
df_zvuk.drop(['timestamp', 'item_id', 'rating'], axis=1)

,user_id,freq_item,user_id_freq_item_max,user_id_freq_item_mean,user_id_freq_item_median,user_id_freq_item_q25,user_id_freq_item_q75
0,6486258,0.000010,0.000730,0.000103,0.000057,0.000025,0.000119
1,7436216,0.000040,0.000891,0.000063,0.000024,0.000010,0.000059
2,1736489,0.000005,0.001046,0.000024,0.000010,0.000005,0.000021
3,4976699,0.000102,0.000467,0.000048,0.000035,0.000010,0.000067
4,4976699,0.000018,0.000467,0.000048,0.000035,0.000010,0.000067
...,...,...,...,...,...,...,...
30957055,3961121,0.000167,0.001097,0.000121,0.000061,0.000024,0.000135
30957056,9067175,0.000014,0.001035,0.000160,0.000068,0.000026,0.000239
30957057,4180391,0.000042,0.001474,0.000172,0.000081,0.000030,0.000218
30957058,2088083,0.000042,0.000889,0.000029,0.000014,0.000005,0.000033


In [ ]:
from  numpy.random import choice

In [ ]:
test = df_zvuk[df_zvuk.user_id.isin(choice(df_zvuk.user_id.unique(), size=100))].reset_index(drop=True)

In [ ]:
test.drop(['freq_item',
           'user_id_freq_item_max',
           'user_id_freq_item_mean',
           'user_id_freq_item_median',
           'user_id_freq_item_q25',
           'user_id_freq_item_q75'], axis=1, inplace=True)

In [ ]:
df_zvuk['mef'] = 2

In [ ]:
p = df_zvuk.groupby(['user_id'])['rating'].mean().rename('job')

In [ ]:
y = df_zvuk.item_id.value_counts()

In [ ]:
import numpy as np

In [ ]:
np.log(1)

0.0

In [ ]:
unique_id = df_zvuk.item_id.unique()
max_id = unique_id.max()
count_id = df_zvuk.item_id.value_counts()

In [ ]:
def formula(df: pd.DataFrame):
  unique_id = df.item_id.unique()
  max_id = unique_id.max()
  count_id = df.item_id.value_counts()

  y = []
  for id_ in range(max_id):
    if id_ in unique_id:
      y.append(count_id[id_])
    else:
      y.append(0)

  y = np.array(y)
  return np.log((y + 0.01)/(y.max() + 0.01))

In [ ]:
f = formula(df_zvuk)

In [ ]:
f.min()

-15.333666386910085

In [ ]:
np.log((0 + 0.01)/(50000 + 0.01))

-15.424948670398354

In [ ]:
y = np.array(y)

In [ ]:
np.log((y + 0.01)/(y.max() + 0.01))

array([-5.04828946, -6.02792493, -7.14469952, ..., -6.96706355,
       -6.37165918, -5.97481981])

In [ ]:
y.max()

0

In [ ]:
unique_id.shape

(106634,)

In [ ]:
df_zvuk.join(p, on = 'item_id', how='cross')

MergeError: Can not pass on, right_on, left_on or set right_index=True or left_index=True

In [ ]:
df_zvuk.user_id.nunique()

161988

In [ ]:
df_zvuk[df_zvuk.user_id.isin(test.user_id.unique())]

,user_id,timestamp,item_id,rating,mef
774,6196938,1673741467559000000,12207,6,2
1225,6196938,1673741827665000000,23421,2,2
1562,6196938,1673742067504000000,58448,5,2
1736,6196938,1673742247484000000,63615,5,2
1927,6196938,1673742427539000000,103186,4,2
...,...,...,...,...,...
30927036,5885423,1682474086384000000,46771,1,2
30928548,5885423,1682474319464000000,3676,1,2
30929809,5885423,1682474508689000000,100711,2,2
30944874,4091280,1682476786434000000,16331,1,2


In [ ]:
test.join(p, on = 'item_id', how = 'inner')

ValueError: columns overlap but no suffix specified: Index(['user_id', 'timestamp', 'item_id', 'rating'], dtype='object')

In [ ]:
p

,user_id,timestamp,item_id,rating,po,obu
0,6196938,1673741467559000000,12207,6,42,6
6,3115102,1673755133529000000,9620,1,42,6
7,6997969,1673755270805000000,97465,1,42,6
9,7882784,1673755544939000000,105903,11,42,6
17,5222718,1673758920867000000,55374,2,42,6
...,...,...,...,...,...,...
14838,9272291,1681189610395000000,40085,1,42,6
16386,5551039,1681873180513000000,3335,1,42,6
16826,4091280,1681997268079000000,68455,1,42,6
17022,2191309,1682086441750000000,50262,1,42,6


In [ ]:
p = test.drop_duplicates(subset='user_id')

In [ ]:
p['po'] = 42

<ipython-input-28-8c05eaf4e934>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p['po'] = 42


In [ ]:
p['obu'] = 6

<ipython-input-31-303f01379872>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p['obu'] = 6


In [ ]:
p.drop(['timestamp	item_id	rating'])

,user_id,timestamp,item_id,rating,po,obu
0,6196938,1673741467559000000,12207,6,42,6
6,3115102,1673755133529000000,9620,1,42,6
7,6997969,1673755270805000000,97465,1,42,6
9,7882784,1673755544939000000,105903,11,42,6
17,5222718,1673758920867000000,55374,2,42,6
...,...,...,...,...,...,...
14838,9272291,1681189610395000000,40085,1,42,6
16386,5551039,1681873180513000000,3335,1,42,6
16826,4091280,1681997268079000000,68455,1,42,6
17022,2191309,1682086441750000000,50262,1,42,6


In [ ]:
test.groupby(['user_id'])['rating']

In [ ]:
test.user_id.value_counts()

,count
user_id,
8708670,1033
8412459,956
7882784,804
1010251,763
1092446,556
...,...
2955556,31
7008685,25
1955987,24


In [ ]:
z = .25
f'p{z}'.replace('0.','')

'p25'

In [ ]:
concat([df_zvuk, feature_groupby(df_zvuk, 'item_id', ['user_id'], 'count')], axis=1)

,user_id,timestamp,item_id,rating,item_id_user_id_count
0,6486258,1673740800074000000,75474,1,323
1,7436216,1673740801368000000,24228,5,1229
2,1736489,1673740801516000000,13113,3,157
3,4976699,1673740803052000000,76029,5,3165
4,4976699,1673740803056000000,105990,1,558
...,...,...,...,...,...
30957055,3961121,1682478698807000000,95865,24,5176
30957056,9067175,1682478698944000000,3615,1,423
30957057,4180391,1682478699230000000,1519,1,1293
30957058,2088083,1682478699233000000,71478,1,1296


In [ ]:
p = df_zvuk.groupby(['item_id'])['user_id'].count()

In [ ]:
p.sort_values()

,user_id
item_id,
24184,1
65399,1
29746,1
24045,1
29734,1
...,...
96036,33948
18700,35268
3065,35966


In [ ]:
l = df_zvuk.item_id.value_counts(normalize=True)

In [ ]:
l = df_zvuk.item_id.value_counts().rename('frq')
df_zvuk.join(l, on = 'item_id', how = 'inner')

,user_id,timestamp,item_id,rating,frq
0,6486258,1673740800074000000,75474,1,323
1,7436216,1673740801368000000,24228,5,1229
2,1736489,1673740801516000000,13113,3,157
3,4976699,1673740803052000000,76029,5,3165
4,4976699,1673740803056000000,105990,1,558
...,...,...,...,...,...
30957055,3961121,1682478698807000000,95865,24,5176
30957056,9067175,1682478698944000000,3615,1,423
30957057,4180391,1682478699230000000,1519,1,1293
30957058,2088083,1682478699233000000,71478,1,1296


In [ ]:
feature_group(df_zvuk, 'item_id', ['user_id'], 'count')

,item_id,user_id,item_id_user_id_count
0,75474,6486258,323
1,24228,7436216,1229
2,13113,1736489,157
3,76029,4976699,3165
4,105990,4976699,558
...,...,...,...
30957055,95865,3961121,5176
30957056,3615,9067175,423
30957057,1519,4180391,1293
30957058,71478,2088083,1296


In [ ]:
l.rename('frq')

,frq
item_id,
44836,1.474236e-03
57480,1.245596e-03
3065,1.161803e-03
18700,1.139255e-03
96036,1.096616e-03
...,...
24045,3.230281e-08
99115,3.230281e-08
29746,3.230281e-08


In [ ]:
l

,proportion
item_id,
44836,1.474236e-03
57480,1.245596e-03
3065,1.161803e-03
18700,1.139255e-03
96036,1.096616e-03
...,...
24045,3.230281e-08
99115,3.230281e-08
29746,3.230281e-08


In [ ]:
df_zvuk

,user_id,timestamp,item_id,rating
0,6486258,1673740800074000000,75474,1
1,7436216,1673740801368000000,24228,5
2,1736489,1673740801516000000,13113,3
3,4976699,1673740803052000000,76029,5
4,4976699,1673740803056000000,105990,1
...,...,...,...,...
30957055,3961121,1682478698807000000,95865,24
30957056,9067175,1682478698944000000,3615,1
30957057,4180391,1682478699230000000,1519,1
30957058,2088083,1682478699233000000,71478,1


In [ ]:
df_zvuk.nunique()

KeyboardInterrupt: 

In [ ]:
md = lambda x: x.mode()[0]
c = 'prtrt'

In [ ]:
p = {'name': 'mode', 'func': lambda x: x.mode()[0]}

In [ ]:
str(p.keys()

dict_keys(['lrmg'])

In [ ]:
tuple(p.items())

(('lrmg', <function __main__.<lambda>(x)>),)

In [ ]:
a

dict_items([('lrmg', <function <lambda> at 0x7c1d02000790>)])

In [ ]:
def timestamp_feature_group(df: pd.DataFrame, column: str, group_columns: str, agg_func: T.Any) -> pd.DataFrame:
  new_name = [f'{column}_{group}_mode' for group in group_columns]
  df[new_name] = df.groupby([column])[group_columns].transform(agg_func)
  return df

md = lambda x: x.mode()[0]

def timestamp_feature(df: pd.DataFrame, column: str):
  new_name = [f'{column}_{group}_mode' for group in group_columns]
  df[new_name] = df.groupby([column])[group_columns].transform(lambda x: x.mode()[0])
  return df

In [ ]:
timestamp_feature(df_zvuk, 'user_id', ['week', 'day_of_week', 'hour'])

,user_id,timestamp,item_id,rating,week,day_of_week,hour,count_hour,user_id_week_mode,user_id_day_of_week_mode,user_id_hour_mode
0,6486258,2023-01-15 00:00:00.074,75474,1,2,6,0,28,3,4,0
1,7436216,2023-01-15 00:00:01.368,24228,5,2,6,0,1,9,4,12
2,1736489,2023-01-15 00:00:01.516,13113,3,2,6,0,46,8,5,20
3,4976699,2023-01-15 00:00:03.052,76029,5,2,6,0,8,14,4,4
4,4976699,2023-01-15 00:00:03.056,105990,1,2,6,0,8,14,4,4
...,...,...,...,...,...,...,...,...,...,...,...
30957055,3961121,2023-04-26 03:11:38.807,95865,24,17,2,3,9,17,2,2
30957056,9067175,2023-04-26 03:11:38.944,3615,1,17,2,3,19,16,2,2
30957057,4180391,2023-04-26 03:11:39.230,1519,1,17,2,3,5,16,6,8
30957058,2088083,2023-04-26 03:11:39.233,71478,1,17,2,3,8,16,3,17


In [ ]:
df_zvuk.drop(['count_hour'], axis=1, inplace=)

,user_id,timestamp,item_id,rating,week,day_of_week,hour,max_count_hour
0,6486258,2023-01-15 00:00:00.074,75474,1,2,6,0,28
1,7436216,2023-01-15 00:00:01.368,24228,5,2,6,0,18
2,1736489,2023-01-15 00:00:01.516,13113,3,2,6,0,80
3,4976699,2023-01-15 00:00:03.052,76029,5,2,6,0,118
4,4976699,2023-01-15 00:00:03.056,105990,1,2,6,0,118
...,...,...,...,...,...,...,...,...
30957055,3961121,2023-04-26 03:11:38.807,95865,24,17,2,3,25
30957056,9067175,2023-04-26 03:11:38.944,3615,1,17,2,3,20
30957057,4180391,2023-04-26 03:11:39.230,1519,1,17,2,3,29
30957058,2088083,2023-04-26 03:11:39.233,71478,1,17,2,3,76


In [ ]:
df_zvuk['week'].mode()

,week
0,16


In [ ]:
df_zvuk['max_count_hour'] = df_zvuk.groupby(['user_id'])['count_hour'].transform('max')

In [ ]:
df_zvuk.weak.value_counts()

,count
weak,
5,4776397
4,4690510
1,4508385
3,4276970
2,4267281
0,4242076
6,4195441


In [ ]:
(df_zvuk.day_of_weak == df_zvuk.weak).value_counts()

,count
True,30957060


In [ ]:
df_zvuk.timestamp.dt.weekday.unique()

array([6, 0, 1, 2, 3, 4, 5], dtype=int32)

In [ ]:
df_zvuk[df_zvuk['user_id'] == df_zvuk['user_id'].unique()[1]]

,user_id,timestamp,item_id,rating,week,day_of_week,hour,count_hour,max_count_hour
1,7436216,2023-01-15 00:00:01.368,24228,5,2,6,0,1,18
207472,7436216,2023-01-15 16:54:08.860,21977,1,2,6,16,9,18
207473,7436216,2023-01-15 16:54:08.915,66759,4,2,6,16,9,18
207474,7436216,2023-01-15 16:54:08.922,43817,1,2,6,16,9,18
207475,7436216,2023-01-15 16:54:08.960,75266,1,2,6,16,9,18
...,...,...,...,...,...,...,...,...,...
13263826,7436216,2023-03-04 20:06:06.453,39029,2,9,5,20,11,18
13471188,7436216,2023-03-05 12:32:08.366,89418,1,9,6,12,18,18
13530031,7436216,2023-03-05 18:08:14.640,3989,2,9,6,18,6,18
14557513,7436216,2023-03-09 12:43:13.921,20431,3,10,3,12,18,18


In [ ]:
df_smm.timestamp.dt.second.unique()

array([ 3, 10, 14, 15, 20, 22, 23, 24, 25, 26, 29, 31, 32, 36, 40, 42, 43,
       53, 57, 59,  0,  2,  8, 16, 27, 34, 35, 41, 45, 48, 49, 51, 58,  6,
       38, 46, 47, 55, 56,  5, 19, 21, 44, 50, 54,  1,  4, 13, 18, 30, 39,
       52,  7,  9, 12, 33, 37, 17, 11, 28], dtype=int32)

In [ ]:
a = df_zvuk[df_zvuk.user_id.isin(df_zvuk_test.user_id.unique())].reset_index()

In [ ]:
df_zvuk

,user_id,timestamp,item_id,rating
0,6486258,1673740800074000000,75474,1
1,7436216,1673740801368000000,24228,5
2,1736489,1673740801516000000,13113,3
3,4976699,1673740803052000000,76029,5
4,4976699,1673740803056000000,105990,1
...,...,...,...,...
30957055,3961121,1682478698807000000,95865,24
30957056,9067175,1682478698944000000,3615,1
30957057,4180391,1682478699230000000,1519,1
30957058,2088083,1682478699233000000,71478,1


In [ ]:
a.user_id.value_counts()

207.1567727345874

In [ ]:
df_zvuk_test.user_id.value_counts()

,count
user_id,
5061075,5638
3819035,5116
4621918,4769
9142952,4432
7334588,4410
...,...
4127855,1
6382914,1
5587307,1


In [ ]:
b = df_zvuk_test.merge(a, indicator=True)

In [ ]:
b._merge.unique()

['both']
Categories (3, object): ['left_only', 'right_only', 'both']

In [ ]:
b.dropna()

,user_id,item_id,timestamp,rating,index,_merge
0,2353151,149597,2023-01-15 00:00:03.033,1.554808,0,both
1,9824229,90964,2023-01-15 00:00:03.105,1.554808,1,both
2,6001776,86260,2023-01-15 00:00:10.376,1.554808,2,both
3,9821399,104024,2023-01-15 00:00:10.924,1.554808,3,both
4,9280991,8232,2023-01-15 00:00:14.425,1.554808,4,both
...,...,...,...,...,...,...
18601319,5017818,36122,2023-04-25 09:14:17.370,1.554808,20689585,both
18601320,8987011,135024,2023-04-25 09:14:17.378,1.554808,20689586,both
18601321,9006098,13386,2023-04-25 09:14:17.383,1.554808,20689587,both
18601322,1738180,154443,2023-04-25 09:14:18.635,1.554808,20689588,both


In [ ]:
b['_merge'].value_counts()

,count
_merge,
both,24458793
left_only,0
right_only,0
